In [13]:
import pandas as pd
import numpy as np
import os
import shutil
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

In [14]:
# Arquivo de origem (demográfico)
SOURCE_FILE = '../database/oasis_longitudinal_demographic.csv'

# Origem das imagens .nii
NII_SOURCE_DIR = '../database/axl'
NII_EXTENSION = '.nii.gz'

# Diretórios de saída
TRAIN_DIR = '../database/treino'
VAL_DIR = '../database/validacion'
TEST_DIR = '../database/teste' 

# Diretórios de saída para imagens NIfTI (.nii)
TRAIN_NII_DEST_DIR = os.path.join(TRAIN_DIR, 'imgs')
VAL_NII_DEST_DIR = os.path.join(VAL_DIR, 'imgs')
TEST_NII_DEST_DIR = os.path.join(TEST_DIR, 'imgs')

# --- Criação dos Diretórios ---
os.makedirs(TRAIN_DIR, exist_ok=True)
os.makedirs(VAL_DIR, exist_ok=True)
os.makedirs(TEST_DIR, exist_ok=True)
os.makedirs(TRAIN_NII_DEST_DIR, exist_ok=True)
os.makedirs(VAL_NII_DEST_DIR, exist_ok=True)
os.makedirs(TEST_NII_DEST_DIR, exist_ok=True)

print(f"Diretórios criados/verificados.")
print(f"- {TRAIN_DIR} (para CSV e Gráficos)")
print(f"- {TRAIN_NII_DEST_DIR} (para imagens .nii)")
print(f"- {VAL_DIR} (para CSV e Gráficos)")
print(f"- {VAL_NII_DEST_DIR} (para imagens .nii)")
print(f"- {TEST_DIR} (para CSV)")
print(f"- {TEST_NII_DEST_DIR} (para imagens .nii)")

Diretórios criados/verificados.
- ../database/treino (para CSV e Gráficos)
- ../database/treino/imgs (para imagens .nii)
- ../database/validacion (para CSV e Gráficos)
- ../database/validacion/imgs (para imagens .nii)
- ../database/teste (para CSV)
- ../database/teste/imgs (para imagens .nii)


In [15]:
try:
    data = pd.read_csv(SOURCE_FILE, sep=';', decimal=',')
except FileNotFoundError:
    print(f"Erro: Arquivo '{SOURCE_FILE}' não encontrado.")
    data = pd.DataFrame()

if not data.empty:
    print(f"\nDados carregados. Forma inicial: {data.shape}")

    # Garante que CDR seja numérico
    if data['CDR'].dtype == 'object':
         data['CDR'] = data['CDR'].str.replace(',', '.').astype(float)
    
    # Define colunas importantes
    target_col = 'Target_Class'
    mri_col = 'MRI ID'

    # Função para aplicar a regra de classificação
    def assign_target_class(row):
        if row['Group'] == 'Demented':
            return 'Demented'
        if row['Group'] == 'Nondemented':
            return 'Nondemented'
        if row['Group'] == 'Converted':
            if row['CDR'] > 0:
                return 'Demented'
            elif row['CDR'] == 0:
                return 'Nondemented'
        return None

    data['Target_Class'] = data.apply(assign_target_class, axis=1)

    # Remove linhas que não puderam ser classificadas (ex: Converted com CDR nulo)
    # ou que não têm as colunas essenciais
    data_processed = data.dropna(
        subset=[target_col, mri_col, 'Age', 'EDUC', 'eTIV', 'nWBV', 'ASF']
    ).copy()
    print(f"Dados após processamento de classes e limpeza. Forma: {data_processed.shape}")

    # --- Definição da Classe do Paciente (para Estratificação) ---
    # Se um paciente teve *qualquer* visita 'Demented', ele é 'Demented'
    patient_class_map = data_processed.groupby('Subject ID')['Target_Class'].apply(
        lambda visits: 'Demented' if 'Demented' in visits.values else 'Nondemented'
    )
    patients_df = pd.DataFrame(patient_class_map).reset_index()
    patients_df.columns = ['Subject ID', 'Patient_Class']

    # --- Divisão Paciente: Treino (64%) / Validação (16%) / Teste (20%) ---
    
    # Treino+Val (80%) vs Teste (20%)
    train_val_patients, test_patients_df = train_test_split(
        patients_df, test_size=0.20, random_state=42, stratify=patients_df['Patient_Class']
    )
    
    # Treino (80% de 80% = 64%) vs Val (20% de 80% = 16%)
    train_patients_df, val_patients_df = train_test_split(
        train_val_patients, test_size=0.20, random_state=42, stratify=train_val_patients['Patient_Class']
    )

    # --- Criação dos DataFrames Finais (Exames) ---
    train_ids = set(train_patients_df['Subject ID'])
    val_ids = set(val_patients_df['Subject ID'])
    test_ids = set(test_patients_df['Subject ID'])

    df_train = data_processed[data_processed['Subject ID'].isin(train_ids)].copy()
    df_val = data_processed[data_processed['Subject ID'].isin(val_ids)].copy()
    df_test = data_processed[data_processed['Subject ID'].isin(test_ids)].copy()

    # --- Salvar CSVs ---
    train_csv_path = os.path.join(TRAIN_DIR, 'train_dataset.csv')
    val_csv_path = os.path.join(VAL_DIR, 'validation_dataset.csv')
    test_csv_path = os.path.join(TEST_DIR, 'test_dataset.csv')
    
    df_train.to_csv(train_csv_path, index=False, sep=';', decimal=',')
    df_val.to_csv(val_csv_path, index=False, sep=';', decimal=',')
    df_test.to_csv(test_csv_path, index=False, sep=';', decimal=',')
    
    print(f"\nDatasets CSV salvos:")
    print(f"- Treino: {train_csv_path} ({len(df_train)} exames)")
    print(f"- Validação: {val_csv_path} ({len(df_val)} exames)")
    print(f"- Teste: {test_csv_path} ({len(df_test)} exames)")

    # --- Cópia dos arquivos .nii (com sufixo _axl) ---

    print("\nIniciando cópia dos arquivos .nii...")

    def copy_nii_files(dataframe, source_dir, dest_dir, extension, mri_id_col):
        if not os.path.isdir(source_dir):
            print(f"!!! AVISO: Diretório de origem NIfTI não encontrado: '{source_dir}'")
            return

        copied_count = 0
        missing_count = 0
        
        for mri_id in dataframe[mri_id_col]:
            # O nome do arquivo é [MRI ID] + "_axl" + [Extensão]
            source_filename = f"{mri_id}_axl{extension}"
            source_path = os.path.join(source_dir, source_filename)
            dest_path = os.path.join(dest_dir, source_filename)
            
            if os.path.exists(source_path):
                try:
                    shutil.copy2(source_path, dest_path)
                    copied_count += 1
                except Exception as e:
                    print(f"Erro ao copiar {source_filename}: {e}")
            else:
                if missing_count < 5:
                    print(f"  Aviso: Arquivo de origem não encontrado: {source_path}")
                missing_count += 1
        
        if missing_count > 5:
            print(f"  ... e mais {missing_count - 5} arquivos não encontrados.")
            
        print(f"Cópia para '{dest_dir}': {copied_count} arquivos copiados, {missing_count} não encontrados na origem.")

    # Executa a cópia para cada conjunto de dados
    copy_nii_files(df_train, NII_SOURCE_DIR, TRAIN_NII_DEST_DIR, NII_EXTENSION, mri_col)
    copy_nii_files(df_val, NII_SOURCE_DIR, VAL_NII_DEST_DIR, NII_EXTENSION, mri_col)
    copy_nii_files(df_test, NII_SOURCE_DIR, TEST_NII_DEST_DIR, NII_EXTENSION, mri_col)


    # --- Geração de Gráficos de Análise (do conjunto de Treino) ---

    print("\nGerando gráficos de análise (Treino)...")
    
    # Distribuição de Classes (Treino)
    plt.figure(figsize=(6, 4))
    sns.countplot(x=target_col, data=df_train, palette='pastel')
    plt.title('Distribuição de Classes (Treino)')
    plt.xlabel('Classe')
    plt.ylabel('Contagem')
    # Salva no diretório raiz de treino
    img_path = os.path.join(TRAIN_DIR, 'train_distribuicao_classes.png')
    plt.savefig(img_path)
    plt.close()
    print(f"- Gráfico salvo: {img_path}")

    # Distribuição de Idade (Treino)
    plt.figure(figsize=(8, 5))
    sns.histplot(x='Age', hue=target_col, data=df_train, kde=True, palette='viridis')
    plt.title('Distribuição de Idade por Classe (Treino)')
    plt.xlabel('Idade')
    plt.ylabel('Contagem')
    # Salva no diretório raiz de treino
    img_path = os.path.join(TRAIN_DIR, 'train_distribuicao_idade.png')
    plt.savefig(img_path)
    plt.close()
    print(f"- Gráfico salvo: {img_path}")
    
    # Distribuição de MMSE (Treino)
    plt.figure(figsize=(8, 5))
    # Filtra nulos ANTES de plotar (só para o gráfico)
    sns.histplot(x='MMSE', hue=target_col, data=df_train.dropna(subset=['MMSE']), kde=True, palette='plasma')
    plt.title('Distribuição de MMSE por Classe (Treino)')
    plt.xlabel('MMSE (Mini-Mental State Examination)')
    plt.ylabel('Contagem')
    # Salva no diretório raiz de treino
    img_path = os.path.join(TRAIN_DIR, 'train_distribuicao_mmse.png')
    plt.savefig(img_path)
    plt.close()
    print(f"- Gráfico salvo: {img_path}")

    print("\n--- Processo de Preparação de Dados Concluído ---")


Dados carregados. Forma inicial: (373, 15)
Dados após processamento de classes e limpeza. Forma: (373, 16)

Datasets CSV salvos:
- Treino: ../database/treino/train_dataset.csv (235 exames)
- Validação: ../database/validacion/validation_dataset.csv (64 exames)
- Teste: ../database/teste/test_dataset.csv (74 exames)

Iniciando cópia dos arquivos .nii...
Cópia para '../database/treino/imgs': 235 arquivos copiados, 0 não encontrados na origem.
Cópia para '../database/validacion/imgs': 64 arquivos copiados, 0 não encontrados na origem.
Cópia para '../database/teste/imgs': 74 arquivos copiados, 0 não encontrados na origem.

Gerando gráficos de análise (Treino)...
- Gráfico salvo: ../database/treino/train_distribuicao_classes.png


/tmp/ipykernel_1554/3086057044.py:129: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.countplot(x=target_col, data=df_train, palette='pastel')


- Gráfico salvo: ../database/treino/train_distribuicao_idade.png
- Gráfico salvo: ../database/treino/train_distribuicao_mmse.png

--- Processo de Preparação de Dados Concluído ---
